In [1]:
import torch
print(torch.__version__)

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

1.7.0
There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 3090


In [2]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
#from tqdm.notebook import trange, tqdm

from uuid import uuid4

## Torch Modules
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [3]:
# loading pre-trained models
from transformers import (
    BertForSequenceClassification,
#     TFBertForSequenceClassification, 
                          BertTokenizer,
#                           TFRobertaForSequenceClassification,
                          RobertaForSequenceClassification,
                          RobertaTokenizer,
                         AdamW)

# BERT
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
                                                                num_labels = 2, # The number of output labels--2 for binary classification.
                                                                                # You can increase this for multi-class tasks.   
                                                                output_attentions = False, # Whether the model returns attentions weights.
                                                                output_hidden_states = False # Whether the model returns all hidden-states.
                                                          )
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# Tell pytorch to run this model on the GPU.
model.cuda()

# RoBERTa
roberta_model = RobertaForSequenceClassification.from_pretrained("roberta-base", # 12-layer, 768-hidden, 12-heads, 125M parameters RoBERTa using the BERT-base architecture
                                                                    num_labels = 2, # The number of output labels--2 for binary classification.
                                                                                    # You can increase this for multi-class tasks.   
                                                                    output_attentions = False, # Whether the model returns attentions weights.
                                                                    output_hidden_states = False # Whether the model returns all hidden-states.
                                                                )
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# Tell pytorch to run this model on the GPU.
roberta_model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [4]:
# Loading weight and biases for the Bert Model
PATH = "bert.pth"

model.load_state_dict(torch.load(PATH))
model.eval()

# Loading weight and biases for the Roberta Model

PATH = "roberta.pth"

roberta_model.load_state_dict(torch.load(PATH))
roberta_model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [29]:
# Text entry and output to input.json
# This is to easily update the input.json
import tkinter as tk
import os
import json

fileout = 'input.json'
   
root=tk.Tk() 
  
# setting the windows size 
root.geometry("225x150") 
   
# declaring string variable 
# for storing input text
text_var=tk.StringVar() 
   
# defining a function that will 
# get the name and password and  
# print them on the screen 
def submit(): 
  
    text=text_entry.get()  
      
    print("Your Text is : " + text) 
      
    text_var.set("") 
    
    # Writing input text to file.
    data = {}
    data['input'] = []
    data['input'].append({
        'body': text})     
        
    with open(fileout, 'w') as f:
        json.dump(data, f, indent=2)
    f.close
    
# defining a function that will close
# the text entry window
def close_window ():
    root.destroy()
      
    
# creating a label for  
# name using widget Label 
text_label = tk.Label(root, text = 'Text',
                      font = ('calibre', 10, 'bold'))
   
# creating a entry for input 
# name using widget Entry 
text_entry = tk.Entry(root, 
                      textvariable = text_var,
                      font=('calibre',10,'normal')) 
   


# creating a button using the widget  
# Button that will call the submit function  
sub_btn=tk.Button(root,text = 'Submit', command = submit )

close_button = tk.Button(root, text = "Close", command = close_window)
   
# placing the label and entry in 
# the required position using grid 
# method 
text_label.grid(row=0,column=0) 
text_entry.grid(row=0,column=1) 
sub_btn.grid(row=2,column=1) 
close_button.grid(row=3,column=1) 
   
# performing an infinite loop  
# for the window to display 
root.mainloop() 

Your Text is :  (CNN)For weeks, experts have warned Americans in increasingly dire terms that this winter will be one of the darkest periods in the nation's public health history.
Now that time is upon us.
From California to Kansas, Massachusetts to Florida, and everywhere in-between, doctors, nurses, funeral home directors and food bank organizers are bracing for a devastating season. They're already stretched thin and exhausted after months of confronting the deadly Covid-19 pandemic, but they fear the worst is still yet to come.


In [26]:
# Loading user input from the input json file
import json

with open('input.json') as f:
    data = json.load(f)

InputText = data['input'][0]['body']

# Display user input
print(InputText)

# Closing file 
f.close() 

 (CNN)For weeks, experts have warned Americans in increasingly dire terms that this winter will be one of the darkest periods in the nation's public health history.
Now that time is upon us.
From California to Kansas, Massachusetts to Florida, and everywhere in-between, doctors, nurses, funeral home directors and food bank organizers are bracing for a devastating season. They're already stretched thin and exhausted after months of confronting the deadly Covid-19 pandemic, but they fear the worst is still yet to come.


In [27]:
# Pad & truncate all input to 500 limit as BERT have a 512 limit.
bert_encoded_input = encoded_input = bert_tokenizer.encode_plus(InputText, 
                                                                max_length = 500,
                                                                truncation = True,
                                                                return_attention_mask = True,
                                                                return_tensors='pt').to(device)
# Prediction for BERT model
bert_result = model(**bert_encoded_input)

# Pad & truncate all input to 500 limit as BERT have a 512 limit.
roberta_encoded_input = encoded_input = roberta_tokenizer.encode_plus(InputText,
                                                                      max_length = 500,
                                                                      truncation = True,
                                                                      return_attention_mask = True,
                                                                      return_tensors='pt').to(device)
# Prediction for RoERTa model
roberta_result = roberta_model(**roberta_encoded_input)

bert_final = bert_result[0]
roberta_final = roberta_result[0]

# Move logits and labels to CPU
bert_final = bert_final.detach().cpu().numpy()
print("BERT returned:")
print(bert_final[0][0],bert_final[0][1])

# Move logits and labels to CPU
roberta_final = roberta_final.detach().cpu().numpy()
print("RoBERTa returned:")
print(roberta_final[0][0], roberta_final[0][1])

BERT returned:
-2.317437 3.0909188
RoBERTa returned:
-3.7170055 4.334596


In [30]:
# Display BERT result and calculate confidence score
result = 0

# The false and true values are compared
# and the higher of the two values is chosen
# confidence score is chosen. 
if bert_final[0][0] > bert_final[0][1]:
    print("BERT rated the statement as False!")
    result = (bert_final[0][0])/5
    printresult = result * 100
    print("BERT have a {:.2f}% Confidence\n".format(printresult))
    
else:
    print("BERT rated the statement as True")
    result = (bert_final[0][1])/5
    printresult = result * 100
    print("BERT have a {:.2f}% Confidence\n".format(printresult))
    
if roberta_final[0][0] > roberta_final[0][1]:
    print("RoBERTa rated the statement as False!")
    result = (roberta_final[0][0])/5
    printresult = result * 100
    print("RoBERTa have a {:.2f}% Confidence\n".format(printresult))
    
else:
    print("RoBERTa rated the statement as True")
    result = (roberta_final[0][1])/5
    printresult = result * 100
    print("RoBERTa have a {:.2f}% Confidence\n".format(printresult))
    
false_result = bert_final[0][0] + roberta_final[0][0]
true_result = bert_final[0][1] + roberta_final[0][1]
# Debug flag
truefalse = 2

if false_result > true_result:
    truefalse = 0
    print("The statement is rated as False!")
    result = (false_result)/10
    printresult = result*100
    print("Models combined have {:.2f}% Confidence".format(printresult))
    
else:
    truefalse = 1
    print("The statement is rated as True!")
    result = (true_result)/10
    printresult = result*100
    print("Models combined have {:.2f}% Confidence".format(printresult))

BERT rated the statement as True
BERT have a 61.82% Confidence

RoBERTa rated the statement as True
RoBERTa have a 86.69% Confidence

The statement is rated as True!
Models combined have 74.26% Confidence
